In [1]:
import time
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
%tensorflow_version 2.x
import tensorflow_datasets as tfds
import tensorflow_hub as hub
from tensorflow.keras import layers
tfds.disable_progress_bar()
tf.enable_eager_execution()

TensorFlow is already loaded. Please restart the runtime to change versions.


In [2]:


# Loading Cats and Dogs Dataset

(train_examples, validation_examples), info = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]', 'train[80%:]'],
    with_info=True,
    as_supervised=True,
)

/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Shuffling and writing examples to /root/tensorflow_datasets/cats_vs_dogs/4.0.0.incompleteDV84CS/cats_vs_dogs-train.tfrecord
Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/4.0.0. Subsequent calls will reuse this data.


In [0]:
# Resizing the images

def format_image(image, label):
  # `hub` image modules exepct their data normalized to the [0,1] range.
  image = tf.image.resize(image, (IMAGE_RES, IMAGE_RES))/255.0
  return  image, label

num_examples = info.splits['train'].num_examples

BATCH_SIZE = 32
IMAGE_RES = 224

train_batches      = train_examples.cache().shuffle(num_examples//4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.cache().map(format_image).batch(BATCH_SIZE).prefetch(1)

In [0]:
# Building a MobileNet model using tensorflow hub.

URL = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
feature_extractor = hub.KerasLayer(URL,input_shape=(IMAGE_RES,IMAGE_RES,3))

In [0]:
# Freezing all the layers in the model.
feature_extractor.trainable = False

In [6]:
# Adding an output layer to the model. 

model = tf.keras.Sequential([
                             feature_extractor,
                             layers.Dense(2,activation=tf.nn.softmax)
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 1280)              2257984   
_________________________________________________________________
dense (Dense)                (None, 2)                 2562      
Total params: 2,260,546
Trainable params: 2,562
Non-trainable params: 2,257,984
_________________________________________________________________


In [7]:
# Training the model

model.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])

EPOCHS = 3
history = model.fit(train_batches,
                    epochs=EPOCHS,
                    validation_data = validation_batches)

Epoch 1/3
582/582 [==============================] - 34s 58ms/step - loss: 0.3358 - acc: 0.9814 - val_loss: 0.0000e+00 - val_acc: 0.0000e+00
Epoch 2/3
582/582 [==============================] - 22s 37ms/step - loss: 0.3245 - acc: 0.9896 - val_loss: 0.3243 - val_acc: 0.9888
Epoch 3/3
582/582 [==============================] - 21s 36ms/step - loss: 0.3223 - acc: 0.9925 - val_loss: 0.3237 - val_acc: 0.9893


In [8]:
# Saving the model as Keras .h5 model.
t = time.time() # Using time so that the name of the file will be a timestamp.
export_path_keras = './{}.h5'.format(int(t))
print(export_path_keras)

model.save(export_path_keras)

./1581940205.h5


In [9]:
!ls

1581940205.h5  sample_data


In [10]:
# Reloading a Keras .h5 model

reloaded = tf.keras.models.load_model(export_path_keras,custom_objects={'KerasLayer':hub.KerasLayer})
reloaded.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 1280)              2257984   
_________________________________________________________________
dense (Dense)                (None, 2)                 2562      
Total params: 2,260,546
Trainable params: 2,562
Non-trainable params: 2,257,984
_________________________________________________________________


In [11]:
# Retraining a saved model

EPOCHS = 3
history = reloaded.fit(train_batches,
                    epochs=EPOCHS,
                    validation_data=validation_batches)

Epoch 1/3
582/582 [==============================] - 25s 42ms/step - loss: 0.3207 - acc: 0.9939 - val_loss: 0.0000e+00 - val_acc: 0.0000e+00
Epoch 2/3
582/582 [==============================] - 21s 37ms/step - loss: 0.3199 - acc: 0.9948 - val_loss: 0.3235 - val_acc: 0.9893
Epoch 3/3
582/582 [==============================] - 21s 37ms/step - loss: 0.3192 - acc: 0.9954 - val_loss: 0.3234 - val_acc: 0.9893


In [12]:
# Saving a model as SavedModel

t = time.time()

export_path_sm = "./{}".format(int(t))
print(export_path_sm)

tf.saved_model.save(model, export_path_sm)


./1581940556
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


INFO:tensorflow:Assets written to: ./1581940556/assets


INFO:tensorflow:Assets written to: ./1581940556/assets


In [13]:
!ls {export_path_sm}

assets	saved_model.pb	variables
